In [8]:
# mongoimport --db newdb --collection restaurants --file primer-dataset.json
!pip install wget
!wget https://raw.githubusercontent.com/mongodb/docs-assets/primer-dataset/primer-dataset.json
!pip install pymongo


Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9680 sha256=79448ea38fbec1bb3669296c68f6adb1616582ab843759eacf3f74f6cbde2b55
  Stored in directory: c:\users\romar\appdata\local\pip\cache\wheels\40\b3\0f\a40dbd1c6861731779f62cc4babcb234387e11d697df70ee97
Successfully built wget


'wget' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.


Defaulting to user installation because normal site-packages is not writeable


In [ ]:
import json
import pymongo
from pymongo import MongoClient
import requests

# Télécharger le fichier de données JSON
url = "https://raw.githubusercontent.com/mongodb/docs-assets/primer-dataset/primer-dataset.json"
response = requests.get(url)
data_file = 'primer-dataset.json'
with open(data_file, 'wb') as file:
    file.write(response.content)

# Créer une connexion à MongoDB
client = MongoClient("mongodb://localhost:27017/")

# Sélectionner la base de données et la collection
db = client["restaurants_state"]
collection = db["restaurants"]

# Importer les données dans MongoDB
with open(data_file, 'r') as file:
    # Charger le JSON en tant que liste de dictionnaires
    data = json.loads("[" + file.read().replace("}\n{", "},\n{") + "]")
    collection.insert_many(data)

# Vérifier l'importation des données
for doc in collection.find({}, {'restaurant': 1, '_id': 0}).limit(5):
    print(doc)

# Remarque : Assurez-vous que le fichier JSON est correctement formaté. S'il contient plusieurs objets JSON non encapsulés dans un tableau, vous devez les formater correctement en un tableau JSON.
# Exportation des données au format JSON
os.system(
    'mongoexport --db newdb --collection restaurants --out datas/json/restaurants.json')


In [ ]:
db.restaurants.find({}, {'name': 1})
db.collection.find({}, {'restaurants': 1, _id: 0})
db.collection.find({}.{"borough": "Manhattan", "name": {"$regex": "^[FDLG].*[0]$"} }


In [29]:
from pymongo import MongoClient

# Créer une connexion à MongoDB
client = MongoClient("mongodb://localhost:27017/")

# Sélectionner la base de données et la collection
db = client["restaurants_state"]
collection = db["restaurants"]

# Obtenir le premier document (restaurant) de la collection
first_restaurant = collection.find_one()

# Extraire le nom et l'adresse
name = first_restaurant['name']
address = first_restaurant['address']

# Afficher le nom et l'adresse
print(f"Nom: {name}")
print("Adresse:")
print(f"  Rue: {address.get('street', 'N/A')}")
print(f"  Code Postal: {address.get('zipcode', 'N/A')}")
print(f"  Arrondissement: {address.get('borough', 'N/A')}")
print(f"  Bâtiment: {address.get('building', 'N/A')}")
print(f"  Coordonnées: {address.get('coord', 'N/A')}")


Nom: Morris Park Bake Shop
Adresse:
  Rue: Morris Park Ave
  Code Postal: 10462
  Arrondissement: N/A
  Bâtiment: 1007
  Coordonnées: [-73.856077, 40.848447]


In [38]:
from pymongo import MongoClient
# regex correspond à toute chaîne qui commence par ‘F’, ‘D’, ‘L’ ou ‘G’ et contient ‘0’ n’importe où dans la chaîne


client = MongoClient('mongodb://localhost:27017/')
filter = {
    'borough': 'Manhattan',
    'name': {
        '$regex': '^[FDLG].*[0]$'
    }
}

result = client['restaurants_state']['restaurants'].find(
    filter=filter
)

# Créer un dictionnaire pour stocker les documents distincts
distinct_documents = {}

# Itérer sur le curseur et ajouter chaque document au dictionnaire
for document in result:
    name = document['name']
    # Si le nom du restaurant n'est pas déjà dans le dictionnaire, ajoutez le document
    if name not in distinct_documents:
        distinct_documents[name] = document

# Itérer sur le dictionnaire et imprimer le nom et l'adresse de chaque document
for name, document in distinct_documents.items():
    address = document['address']
    print(f"Nom du restaurant : {name}")
    print(f"Adresse : {address['building']} {
          address['street']}, {address['zipcode']}")


Nom du restaurant : Fashion 40
Adresse : 202 West   40 Street, 10018


In [39]:
from pymongo import MongoClient

# nom de restaurant de Manhattan comportant au moins deux voyelles,

client = MongoClient('mongodb://localhost:27017/')
filter = {
    'borough': 'Manhattan',
    'name': {
        '$regex': '^.*[aeiouAEIOU].*[aeiouAEIOU].*$',
        '$options': 'i'
    }
}

result = client['restaurants_state']['restaurants'].find(
    filter=filter
)

# Itérer sur le curseur et imprimer chaque document
for document in result:
    name = document['name']
    address = document['address']
    print(f"Nom du restaurant : {name}")
    print(f"Adresse : {address['building']} {
          address['street']}, {address['zipcode']}")


Nom du restaurant : Dj Reynolds Pub And Restaurant
Adresse : 351 West   57 Street, 10019
Nom du restaurant : 1 East 66Th Street Kitchen
Adresse : 1 East   66 Street, 10065
Nom du restaurant : Glorious Food
Adresse : 522 East   74 Street, 10021
Nom du restaurant : Bully'S Deli
Adresse : 759 Broadway, 10003
Nom du restaurant : Harriet'S Kitchen
Adresse : 502 Amsterdam Avenue, 10024
Nom du restaurant : P & S Deli Grocery
Adresse : 730 Columbus Avenue, 10025
Nom du restaurant : Angelika Film Center
Adresse : 18 West Houston Street, 10012
Nom du restaurant : The Country Cafe
Adresse : 60 Wall Street, 10005
Nom du restaurant : Downtown Deli
Adresse : 107 Church Street, 10007
Nom du restaurant : Olive'S
Adresse : 120 Prince Street, 10012
Nom du restaurant : Cafe Metro
Adresse : 625 8 Avenue, 10018
Nom du restaurant : Lexler Deli
Adresse : 405 Lexington Avenue, 10174
Nom du restaurant : Lorenzo & Maria'S
Adresse : 1418 Third Avenue, 10028
Nom du restaurant : Domino'S Pizza
Adresse : 464 3 Aven

In [40]:
from pymongo import MongoClient

# nom de restaurant de Manhattant comportant au moins deux voyelles, et qui commence par f ou p et terminer par la lettre i ou e

client = MongoClient('mongodb://localhost:27017/')
filter = {
    'borough': 'Manhattan',
    'name': {
        '$regex': '^(f|p).*[aeiouAEIOU].*[aeiouAEIOU].*(i|e)$',
        '$options': 'i'
    }
}

result = client['restaurants_state']['restaurants'].find(
    filter=filter
)

# Itérer sur le curseur et imprimer chaque document
for document in result:
    name = document['name']
    address = document['address']
    print(f"Nom du restaurant : {name}")
    print(f"Adresse : {address['building']} {
          address['street']}, {address['zipcode']}")


Nom du restaurant : Finnegan'S Wake
Adresse : 1361 First Avenue, 10021
Nom du restaurant : Flame Restaurant Coffee House
Adresse : 893 9 Avenue, 10019
Nom du restaurant : Frankie & Johnnies Steakhouse
Adresse : 269 West 45 Street, 10036
Nom du restaurant : Periyali
Adresse : 35 West   20 Street, 10011
Nom du restaurant : Pomodoro Ristorante
Adresse : 51 Spring Street, 10012
Nom du restaurant : Pomodoro Ristorante
Adresse : 229 Columbus Avenue, 10023
Nom du restaurant : Peking Duck House
Adresse : 236 East   53 Street, 10022
Nom du restaurant : Firehouse
Adresse : 522 Columbus Avenue, 10024
Nom du restaurant : Paddy Maguire'S Ale House
Adresse : 237 Third Avenue, 10003
Nom du restaurant : Food Mart Deli
Adresse : 1321 York Avenue, 10021
Nom du restaurant : Pongal Indian Cuisine
Adresse : 110 Lexington Avenue, 10016
Nom du restaurant : Post Billiards Cafe
Adresse : 154 Post Avenue, 10034
Nom du restaurant : Fireside
Adresse : 21 East   52 Street, 10022
Nom du restaurant : Paris Cafe
Adre

In [ ]:
# commente  en fancais  en commencant par des verbes à l'infinitif:
# Décomposer chaque élément de l’array “grades” dans des documents séparés. Chaque document résultant remplace l’array par un élément de l’array.
varUnwind = {$unwind : "$grades"}
# Regrouper les documents par “borough” et calculer la moyenne des scores de “grades” pour chaque groupe.
varGroup4 = { $group : {"_id" : "$borough", "moyenne" : {$avg : "$grades.score"} } };
# Trier les documents par la moyenne des scores en ordre décroissant.
varSort2 = { $sort : { "moyenne" : -1 } }
#  pipeline d’agrégation dans MongoDB, série d’étapes où chaque étape transforme les documents au fur et à mesure de leur passage dans le pipeline
db.restaurants.aggregate( [ varUnwind, varGroup4, varSort2 ] );
# $unwind déconstruit un champ de tableau des documents d’entrée pour produire un document pour chaque élément.
# $group regroupe les documents d’entrée par une expression d’identifiant spécifiée et applique l’expression(s) d’accumulateur, si spécifiée, à chaque groupe.
# $sort trie les documents d’entrée.


In [ ]:
# Compter le nombre de restaurants avec un grade A :

db.restaurants.countDocuments({"grades.grade": "A"})


In [ ]:
# Trier tous les différents scores de restaurants en ordre décroissant :
db.restaurants.find({}).sort({"grades.score": -1})


In [ ]:
# Trouver tous les restaurants où les noms des villes commencent par B, C, D ou se terminent par une voyelle sauf y
db.restaurants.find({
  $or: [
    { "borough": { $regex: /^[BCD]/ } },
    { "borough": { $regex: /[AEIOU]$/, $options: 'i' } }
  ]
})


In [ ]:
# Afficher tous les restaurants si et seulement si le score est inférieur à 20 ou égal à 25, 30, 35, et 40
db.restaurants.find({
  "grades.score": {
    $in: [25, 30, 35, 40],
    $not: { $gt: 20 }
  }
})


In [ ]:
# Faire la somme du nombre de restaurants par type de cuisine :
db.restaurants.aggregate([
  { $group: { _id: "$cuisine", count: { $sum: 1 } } }
])


In [ ]:
# Ajouter le commentaire “Je gère le NoSQL” pour les boroughs dont le nom commence par “B” :
db.restaurants.updateMany(
  { "borough": { $regex: /^B/ } },
  { $set: { "comments": "Je gère le NoSQL" } }
)


In [ ]:
# Supprimer la clé “address” des restaurants qui ont un score supérieur à 25 :
db.restaurants.updateMany(
  { "grades.score": { $gt: 25 } },
  { $unset: { "address": "" } }
)


In [ ]:
# Supprimer tous les restaurants dont le nom de quartier est “Queens” :
db.restaurants.deleteMany({"borough": "Queens"})
